# Data collation and analysis
This file handles collecting all collected data into one consistent table.

## Data Collection
First data is collected, then any non-gaia stellar IDs (such as HD numbers) are matched 
with a corresponding Gaia star entry. 

## Analysis
After data has been collected into a common table, mass analysis can be performed

In [18]:
import pandas as pd
import numpy as np
from astropy.table import Table
from astroquery.gaia import Gaia

Gaia.MAIN_GAIA_TABLE = "gaiadr3.gaia_source"  # Select Data Release 3, default

## Log in to Gaia
Use credentials from gaia/CREDENTIALS file

In [19]:
Gaia.login(credentials_file='gaia/CREDENTIALS')
username = 'mwidmaie'

INFO: Login to gaia TAP server [astroquery.gaia.core]
OK
INFO: Login to gaia data server [astroquery.gaia.core]
OK


## Radii Data
The initial radii dataset was insufficient, so we need to use another dataset and combine them into one. 
We will use the data from `data/stellar_ldd.vot` as well as some Gaia data to manually calculate the stellar radii (using trigonometry).

In [4]:
stellar_ldd_table_name = 'user_' + username + ".stellar_ldd"

try:
    stellar_ldd = Gaia.load_table(stellar_ldd_table_name)
except:
# Table doesn't exist so we're gonna upload from the local stellar_ldd.vot file
print("Table does not exist, uploading from local machine.")
Gaia.upload_table(upload_resource="data/stellar_ldd.vot", table_name='stellar_ldd', format="VOTable")
stellar_ldd = Gaia.load_table(stellar_ldd_table_name)

Retrieving table 'user_mwidmaie.stellar_ldd'
500 Error 500:
esavo.tap.TAPException: Code: 404, msg: Table 'user_mwidmaie.stellar_ldd' not found.
Table does not exist, uploading from local machine.
Sending file: data/stellar_ldd.vot
Uploaded table 'stellar_ldd'.
Retrieving table 'user_mwidmaie.stellar_ldd'


## Update metadata for stellar_ldd
This will make the next steps a lot easier. (and faster)

In [5]:
    Gaia.update_user_table(table_name=stellar_ldd_table_name,
                           list_of_changes=[['"RAJ2000"', 'flags', 'Ra'], ['"DEJ2000"', 'flags', 'Dec'],
                                            ['"RAJ2000"', 'indexed', True], ['"DEJ2000"', 'indexed', True]])

Retrieving table 'user_mwidmaie.stellar_ldd'
Table 'user_mwidmaie.stellar_ldd' updated.


## Retrieving Gaia Parallax and Parallax Error
We need to retrieve the parallax and parallax error from Gaia for each star in the stellar_ldd table.

With this data, we can calculate the distance to the star, and then use the angular diameter from the stellar_ldd table to calculate the stellar radius.
We can combine the data using the following ADQL query:
```sql
SELECT recno, source_id, Name, RAJ2000, DEJ2000, LDD, e_LDD, ra as gaia_ra, dec as gaia_dec, vmag, parallax, parallax_error,
DISTANCE(
    POINT(RAJ2000, DEJ2000),
    POINT(ra, dec)
) * 3600. AS dist_arcsec
FROM user_<username>.stellar_ldd AS stellar_ldd
JOIN gaiadr3.gaia_source AS gaia
-- Geometric Cross-Match: =======
ON DISTANCE(
    POINT(RAJ2000, DEJ2000),
    POINT(ra, dec)
) < 1.8 / 3600.
-- Condition: ===================
WHERE ABS(vmag - phot_g_mean_mag) < 2.
```

Processing will be done in Python.

In [6]:
import gzip

job = Gaia.launch_job(f"""SELECT TOP 466000 recno, source_id, Name, RAJ2000, DEJ2000, ra as gaia_ra, dec as gaia_dec, vmag, parallax, parallax_error, LDD, e_LDD,
DISTANCE(
    POINT(RAJ2000, DEJ2000),
    POINT(ra, dec)
) * 3600. AS dist_arcsec
FROM {stellar_ldd_table_name} AS stellar_ldd
JOIN gaiadr3.gaia_source AS gaia
-- Geometric Cross-Match: =======
ON DISTANCE(
    POINT(RAJ2000, DEJ2000),
    POINT(ra, dec)
) < 1.8 / 3600.
-- Condition: ===================
WHERE ABS(vmag - phot_g_mean_mag) < 2.""", dump_to_file=True, output_format='votable',
                      output_file='data/stellar_ldd_gaia.vot.gz', verbose=True)

results = job.get_results()

with gzip.open('data/stellar_ldd_gaia.vot.gz', 'rb') as f:
    with open('data/stellar_ldd_gaia.vot', 'wb') as f2:
        f2.write(f.read())

Launched query: 'SELECT TOP 466000 recno, source_id, Name, RAJ2000, DEJ2000, ra as gaia_ra, dec as gaia_dec, vmag, parallax, parallax_error, LDD, e_LDD,
DISTANCE(
    POINT(RAJ2000, DEJ2000),
    POINT(ra, dec)
) * 3600. AS dist_arcsec
FROM user_mwidmaie.stellar_ldd AS stellar_ldd
JOIN gaiadr3.gaia_source AS gaia
-- Geometric Cross-Match: =======
ON DISTANCE(
    POINT(RAJ2000, DEJ2000),
    POINT(ra, dec)
) < 1.8 / 3600.
-- Condition: ===================
WHERE ABS(vmag - phot_g_mean_mag) < 2.'
------>https
host = gea.esac.esa.int:443
context = /tap-server/tap/sync
Content-type = application/x-www-form-urlencoded
200 200
[('Date', 'Thu, 18 Jan 2024 17:13:24 GMT'), ('Server', 'Apache/2.4.6 (CentOS) OpenSSL/1.0.2k-fips mod_jk/1.2.43'), ('Cache-Control', 'no-cache, no-store, max-age=0, must-revalidate'), ('Pragma', 'no-cache'), ('Expires', '0'), ('X-XSS-Protection', '1; mode=block'), ('X-Frame-Options', 'SAMEORIGIN'), ('X-Content-Type-Options', 'nosniff'), ('Content-Encoding', 'gzip'), ('

## Calculating Stellar Radii
### Calculating Distance
We can calculate the distance to the star using the parallax from Gaia, using the following formula:
![Distance Formula](imgs/latex_distance.png)
Where:
d = distance to star (in parsecs)
p = parallax angle (in mas)

### Calculating Stellar Radius
We can calculate the stellar radius using the angular diameter from the stellar_ldd table, using the following formula:
![Stellar Radius Formula](imgs/latex_radius.png)
Where:
r_star = stellar radius (in kilometers)
d = distance to star (in parsecs)
theta = angular diameter (in radians)

### Calculating Stellar Radius Error
We can calculate the stellar radius error using the following formula:
![Stellar Radius Error Formula](imgs/latex_radius_error.png)

In [7]:
import numpy as np

df = Table.read('data/stellar_ldd_gaia.vot').to_pandas()
df['distance'] = 1000 / df['parallax']  # Convert parallax to distance in km
df['LDD_rad'] = df[
                    'LDD'] / 1000 / 3600 * np.pi / 180  # Convert LDD to radians (First convert to arcseconds, then to degrees, then to radians)
df['e_LDD_rad'] = df['e_LDD'] / 1000 / 3600 * np.pi / 180  # Convert e_LDD to radians
df['radius'] = 1 / 2 * df['distance'] * np.tan(df['LDD_rad']) * 3.086e+13  # Calculate radius in km 
df['distance_error'] = 1000 / np.square(df['parallax']) * df['parallax_error']  # Calculate distance error in parsecs
df['distance_error_ratio'] = df['distance_error'] / df['distance']  # Calculate distance error as a ratio
df['radius_error'] = np.sqrt(
    np.square(1 / 2 * df['distance'] * 1 / (np.square(np.cos(df['LDD_rad']))) * df['e_LDD_rad']) + np.square(
        1 / 2 * df['distance_error'] * df['parallax_error'] * np.tan(
            df['LDD_rad']))) * 3.086e+13  # Calculate radius error in km
df['radius_error_ratio'] = df['radius_error'] / df['radius']  # Calculate radius error as a ratio

df['radius'] = df['radius'] / 695700  # Convert radius to solar radii
df['radius_error'] = df['radius_error'] / 695700  # Convert radius error to solar radii

df.drop(columns=['LDD_rad', 'e_LDD_rad'], inplace=True)
df.dropna(subset=['distance', 'distance_error', 'radius', 'radius_error', 'source_id', 'gaia_dec', 'gaia_ra'],
          inplace=True)
df.drop(columns=['parallax', 'parallax_error', 'LDD', 'e_LDD', 'RAJ2000', 'DEJ2000', 'Name', 'dist_arcsec', 'vmag',
                 'distance', 'distance_error', 'distance_error_ratio', 'recno'], inplace=True)
df

,source_id,gaia_ra,gaia_dec,radius,radius_error,radius_error_ratio
0,7632157690368,45.034343,0.235390,5.713006,0.127737,0.022359
1,16733192740608,45.152965,0.386342,1.489124,0.040466,0.027174
2,30343944744320,45.094992,0.476836,3.069619,0.085569,0.027876
3,44358422235136,45.501454,0.497697,1.223615,0.030880,0.025237
4,64531884461952,45.490906,0.741372,29.043697,0.770821,0.026540
...,...,...,...,...,...,...
327676,5310910375029195520,137.302009,-55.287044,4.171335,0.098949,0.023721
327677,5310927108226060160,137.016883,-55.176849,5.069081,0.139604,0.027540
327678,5310928482615564672,136.919700,-55.108573,4.154725,0.115875,0.027890
327679,5310928688774038656,137.041132,-55.099785,2.860295,0.093900,0.032829


In [9]:
# Save as a VOTable and upload to Gaia
stellar_radii_table_name = 'user_' + username + ".stellar_radii"
stellar_radii_vot = Table.from_pandas(df)
stellar_radii_vot.write('data/stellar_radii_finalized.vot', format='votable', overwrite=True)

Gaia.upload_table(upload_resource='data/stellar_radii_finalized.vot', table_name='stellar_radii', format='votable')
Gaia.update_user_table(table_name=stellar_radii_table_name,
                       list_of_changes=[['gaia_ra', 'flags', 'Ra'], ['gaia_dec', 'flags', 'Dec'],
                                        ['gaia_ra', 'indexed', True], ['gaia_dec', 'indexed', True]])
Gaia.delete_user_table(table_name=stellar_ldd_table_name)  # Delete the old stellar_ldd table

Sending file: data/stellar_radii_finalized.vot
Uploaded table 'stellar_radii'.
Retrieving table 'user_mwidmaie.stellar_radii'
Table 'user_mwidmaie.stellar_radii' updated.
Table 'user_mwidmaie.stellar_ldd' deleted.


## Collating with stellar mass data
Using data from `data/stellar_radii_combined.vot` we can pair up the Gaia source ids from `data/stellar_masses.rawdata` to form a new dataset including both radii and mass, 2 of the 3 data points we need to calculate stellar core temperature

In [10]:
# We need to download and pre-process the data from https://cdsarc.cds.unistra.fr/viz-bin/nph-Cat/txt?I/360/binmass.dat.gz
# This is a table of stellar masses
import requests

url = 'https://cdsarc.cds.unistra.fr/viz-bin/nph-Cat/txt?I/360/binmass.dat'
print("Downloading data from " + url + "... This may take a while.")
r = requests.get(url, allow_redirects=True)

with open('data/binmass.rawdata', 'wb') as f:
    f.write(r.content)

print('Download complete, processing...')
fixed_content = ''
# Now we need to process the data, since it's not in a standard format
with open('data/binmass.rawdata', 'r') as f:
    lines = f.readlines()[4:-1]
    for line in lines:
        if line[0] == '#' or line[0] == '-':
            continue
        content = line.replace('|', ',')
        ra_and_dec = content.split(',')[13]
        ra = ra_and_dec.split(' ')[0]
        dec = ra_and_dec.split(' ')[-1]
        content = content.replace(ra_and_dec, ra + ',' + dec).replace(' ', '')
        fixed_content += content

with open('data/stellar_masses.rawdata', 'w') as f:
    f.write(fixed_content)

print('Done')

Download complete, processing...
Done


## Uploading stellar_masses to Gaia
We need to upload the stellar_masses data to Gaia so we can join it with the stellar_radii data.

In [11]:
# Load stellar_masses.rawdata as a Pandas DataFrame

# Removed the column headers because they weren't specific enough, we'll use these column names instead
columns = ['source_id', 'mass_primary', 'mass_primary_lower', 'mass_primary_upper', 'mass_secondary',
           'mass_secondary_lower', 'mass_secondary_upper', 'flux_ratio', 'flux_ratio_lower',
           'flux_ratio_upper', 'method', 'reference_for_primary_mass', 'flag', 'ra2016', 'dec2016']

# Load the data into a Pandas DataFrame
stellar_masses = pd.read_csv('data/stellar_masses.rawdata', names=columns, skiprows=4)
stellar_masses.drop(
    columns=['mass_secondary_lower', 'mass_secondary_upper', 'flux_ratio_lower', 'flux_ratio_upper', 'flag', 'method',
             'reference_for_primary_mass'], inplace=True)

# Drop any with NaN ra or dec
stellar_masses.dropna(subset=['ra2016', 'dec2016'], inplace=True)

# Save as a VOTable
stellar_masses_vot = Table.from_pandas(stellar_masses)
stellar_masses_vot.write('data/stellar_masses.vot', format='votable', overwrite=True)

# Upload to Gaia
Gaia.upload_table(upload_resource='data/stellar_masses.vot', table_name='stellar_masses', format='votable')

Sending file: data/stellar_masses.vot
Uploaded table 'stellar_masses'.


## Joining stellar_masses and stellar_radii

We can join the stellar_masses and stellar_radii tables using the following ADQL query:
```sql
SELECT TOP 150000 stellar_radii.source_id, gaia_ra as ra, gaia_dec as dec, radius, radius_error, mass_primary, mass_secondary
FROM {stellar_radii_table_name} AS stellar_radii
JOIN {stellar_masses_table_name} AS stellar_masses
ON stellar_radii.source_id = stellar_masses.source_id
```

In [13]:
import gzip

stellar_radii_table_name = 'user_' + username + ".stellar_radii"
stellar_masses_table_name = 'user_' + username + ".stellar_masses"
combined_table_name = 'user_' + username + ".stellar_masses_and_radii"

try:
    stellar_mass_and_radii = Gaia.load_table(combined_table_name)
except:
    # Table doesn't exist so we're gonna upload from the local stellar_ldd.vot file
    print("Table does not exist, running queries and uploading from local machine.")
    job = Gaia.launch_job(f"""SELECT TOP 150000 stellar_radii.source_id, gaia_ra as ra, gaia_dec as dec, radius, radius_error, mass_primary, mass_secondary, mass_primary_lower, mass_primary_upper
FROM {stellar_radii_table_name} AS stellar_radii
JOIN {stellar_masses_table_name} AS stellar_masses
ON stellar_radii.source_id = stellar_masses.source_id""", dump_to_file=True, output_format='votable',
                          output_file='data/stellar_masses_and_radii.vot.gz', verbose=True)

    results = job.get_results()

    with gzip.open('data/stellar_masses_and_radii.vot.gz', 'rb') as f:
        with open('data/stellar_masses_and_radii.vot', 'wb') as f2:
            f2.write(f.read())

    table = Table.read('data/stellar_masses_and_radii.vot')
    # Set units for the table
    table['radius'].unit = 'Rsun'
    table['radius_error'].unit = 'Rsun'
    table['mass_primary'].unit = 'Msun'
    table['mass_secondary'].unit = 'Msun'
    table['mass_primary_lower'].unit = 'Msun'
    table['mass_primary_upper'].unit = 'Msun'
    table['ra'].unit = 'deg'
    table['dec'].unit = 'deg'

    # Save as a VOTable and upload to Gaia
    table.write('data/stellar_masses_and_radii.vot', format='votable', overwrite=True)

    Gaia.upload_table(upload_resource='data/stellar_masses_and_radii.vot', table_name='stellar_masses_and_radii',
                      format='votable')
    Gaia.update_user_table(table_name=combined_table_name,
                           list_of_changes=[['ra', 'flags', 'Ra'], ['dec', 'flags', 'Dec'], ['ra', 'indexed', True],
                                            ['dec', 'indexed', True]])

    # Remove the old tables
    Gaia.delete_user_table(table_name=stellar_radii_table_name)
    Gaia.delete_user_table(table_name=stellar_masses_table_name)

Retrieving table 'user_mwidmaie.stellar_masses_and_radii'
500 Error 500:
esavo.tap.TAPException: Code: 404, msg: Table 'user_mwidmaie.stellar_masses_and_radii' not found.
Table does not exist, running queries and uploading from local machine.
Launched query: 'SELECT TOP 150000 stellar_radii.source_id, gaia_ra as ra, gaia_dec as dec, radius, radius_error, mass_primary, mass_secondary, mass_primary_lower, mass_primary_upper
FROM user_mwidmaie.stellar_radii AS stellar_radii
JOIN user_mwidmaie.stellar_masses AS stellar_masses
ON stellar_radii.source_id = stellar_masses.source_id'
------>https
host = gea.esac.esa.int:443
context = /tap-server/tap/sync
Content-type = application/x-www-form-urlencoded
200 200
[('Date', 'Thu, 18 Jan 2024 17:35:02 GMT'), ('Server', 'Apache/2.4.6 (CentOS) OpenSSL/1.0.2k-fips mod_jk/1.2.43'), ('Cache-Control', 'no-cache, no-store, max-age=0, must-revalidate'), ('Pragma', 'no-cache'), ('Expires', '0'), ('X-XSS-Protection', '1; mode=block'), ('X-Frame-Options', 'SA

Sending file: data/stellar_masses_and_radii.vot
Uploaded table 'stellar_masses_and_radii'.
Retrieving table 'user_mwidmaie.stellar_masses_and_radii'
Table 'user_mwidmaie.stellar_masses_and_radii' updated.
Table 'user_mwidmaie.stellar_radii' deleted.
Table 'user_mwidmaie.stellar_masses' deleted.


## Calculating Stellar Core Temperature
We can calculate the stellar core temperature using a generalized formula based on the Lane-Emden equation: (Solved using the Sun's data as a reference)

According to [this paper](https://jila.colorado.edu/~ajsh/courses/astr1120_03/text/chapter5/Hydrostatic.htm), this equation works as a good generalization for all main sequence stars.

The formula is as follows:
![Stellar Core Temperature Formula](imgs/latex_core_temp.png)
Where:
`T sub c` = Stellar core temperature (in Kelvin)
`M sub *` = Stellar mass (in solar masses)
`R sub *` = Stellar radius (in radii)

We will use pandas to calculate the core temperature for each star in the dataset.

In [27]:
import numpy as np
# Load stellar_masses_and_radii.vot as a Pandas DataFrame
combined_table_name = 'user_' + username + ".stellar_masses_and_radii"
combined_table: Table = Table.read('data/stellar_masses_and_radii.vot')
combined_table_df = combined_table.to_pandas(index='source_id')

# Calculate core temperature
combined_table_df['core_temp'] = 1.5e+7 * combined_table_df['mass_primary'] / np.power(combined_table_df['radius'], -1)

## Calculating Stellar Core Temperature Error
Now that we have the core temperature, we must calculate the error. This is a simple procedure relying on error propagation with multiplied values (`M sub *` and `R sub *`)

The formula is as follows:
![Stellar Core Temperature Error Formula](imgs/latex_core_temp_error.png)

Where:
`ΔT sub c` = Absolute stellar core temperature error (in Kelvin)
`T sub c` = Stellar core temperature (in Kelvin)
`ΔM sub *` = Stellar mass error (in solar masses), calculated using the following formula:
![Stellar Mass Error Formula](imgs/latex_mass_error.png)

Where:
`ΔM sub *` = Stellar mass error (in solar masses)
`M sub upper` = Upper stellar mass (in solar masses), Given
`M sub lower` = Lower stellar mass (in solar masses), Given

`ΔR sub *` = Stellar radius error (in solar radii), Given
`M sub avg` = Average of upper and lower stellar masses (in solar masses)
`R sub *` = Stellar radius (in solar radii), Given

We will use pandas to calculate the core temperature error for each star in the dataset.

In [34]:
# Calculate core temperature error
combined_table_df['mass_error'] = np.abs(combined_table_df['mass_primary_upper'] - combined_table_df['mass_primary_lower']) / 2
combined_table_df['mass_avg'] = (combined_table_df['mass_primary_upper'] + combined_table_df['mass_primary_lower']) / 2

combined_table_df['core_temp_rel_error'] = np.sqrt(np.square(combined_table_df['mass_error'] / combined_table_df['mass_avg']) + np.square(-1 * combined_table_df['radius_error']))
combined_table_df['core_temp_abs_error'] = combined_table_df['core_temp'] * combined_table_df['core_temp_rel_error']

combined_table_df

,ra,dec,radius,radius_error,mass_primary,mass_secondary,mass_primary_lower,mass_primary_upper,core_temp,core_temp_abs_error,core_temp_rel_error,mass_error,mass_avg
source_id,,,,,,,,,,,,,
577600125523574400,133.654404,1.450089,1.070391,0.028464,1.093606,NaN,1.036624,1.149162,1.755879e+07,1.032992e+06,0.058831,0.056269,1.092893
578724303147997824,138.139429,2.688561,1.881674,0.048006,1.383444,NaN,1.199296,1.435773,3.904787e+07,3.974123e+06,0.101776,0.118239,1.317534
579463965236154112,140.505721,4.806395,1.517894,0.047218,1.052349,1.037993,0.990182,1.111813,2.396032e+07,1.789469e+06,0.074685,0.060815,1.050998
579840685407304704,136.499234,3.674005,1.697662,0.041493,1.334517,NaN,1.281419,1.389256,3.398336e+07,1.967533e+06,0.057897,0.053919,1.335338
580223869504212480,135.398586,4.986870,3.971047,0.118664,2.108106,NaN,2.043757,2.164858,1.255708e+08,1.533250e+07,0.122102,0.060550,2.104308
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5310291624866429952,138.681301,-54.788978,2.560037,0.071639,2.012350,NaN,1.943559,2.077520,7.727537e+07,6.105216e+06,0.079006,0.066980,2.010540
5310329351859038336,140.457997,-55.514349,3.178390,0.094913,2.050419,NaN,1.794378,2.112532,9.775546e+07,1.222526e+07,0.125060,0.159077,1.953455
5310329351859038336,140.457997,-55.514349,3.178390,0.094913,1.948070,2.332806,1.853917,2.056918,9.287592e+07,1.004729e+07,0.108180,0.101501,1.955417


### Save to Gaia
Export to VOT, update labels then push to Gaia

In [40]:
# combined_table_df.drop(columns=['mass_error', 'mass_avg'], inplace=True)
# Drop data with a temperature error greater than Q3 + 1.5 * IQR
q3 = combined_table_df['core_temp_rel_error'].quantile(0.75)
iqr = q3 - combined_table_df['core_temp_rel_error'].quantile(0.25)
outlier = q3 + 1.5 * iqr
combined_table_df = combined_table_df[combined_table_df['core_temp_rel_error'] < outlier]

table: Table = Table.from_pandas(combined_table_df, index='source_id')

# Update labels
table['source_id'].description = 'Gaia Source ID'
table['ra'].description = 'Right Ascension'
table['dec'].description = 'Declination'
table['radius'].description = 'Stellar Radius'
table['radius_error'].description = 'Stellar Radius Error'
table['mass_primary'].description = 'Primary Stellar Mass'
table['mass_secondary'].description = 'Secondary Stellar Mass'
table['core_temp'].description = 'Stellar Core Temperature'
table['core_temp_abs_error'].description = 'Absolute Stellar Core Temperature Error'

# Update units
table['ra'].unit = 'deg'
table['dec'].unit = 'deg'
table['radius'].unit = 'Rsun'
table['radius_error'].unit = 'Rsun'
table['mass_primary'].unit = 'Msun'
table['mass_secondary'].unit = 'Msun'
table['core_temp'].unit = 'K'
table['core_temp_abs_error'].unit = 'K'

table.meta.description = 'Final dataset of Stellar Masses, Radii, and Core Temperatures'
table.meta.keywords = ['Stellar', 'Mass', 'Radius', 'Core Temperature', 'Gaia']
table.meta.curation = 'Data collated from various sources, including Gaia and Vizier, and processed using Python and Astroquery.'
table.meta.curation_date = '2024-01-18'
table.meta.creator_name = 'Maximus Widmaier'

# Save as a VOTable and upload to Gaia
table.write('data/stellar_dataset_final.vot', format='votable', overwrite=True, table_id='stellar_dataset_final')

final_table_name = 'user_' + username + ".stellar_dataset_final"

Gaia.upload_table(upload_resource='data/stellar_dataset_final.vot', table_name='stellar_dataset_final', format='votable')

Gaia.update_user_table(table_name=final_table_name,
                       list_of_changes=[['ra', 'flags', 'Ra'], ['dec', 'flags', 'Dec'], ['ra', 'indexed', True],
                                        ['dec', 'indexed', True]])

# Delete the old tables
Gaia.delete_user_table(table_name=combined_table_name)

Sending file: data/stellar_dataset_final.vot
Uploaded table 'stellar_dataset_final'.
Retrieving table 'user_mwidmaie.stellar_dataset_final'
Table 'user_mwidmaie.stellar_dataset_final' updated.
500 Error 500:
Can not execute the following SQL: 
DROP TABLE user_mwidmaie.stellar_masses_and_radii
Because: ERROR: table "stellar_masses_and_radii" does not exist


HTTPError: Error 500:
Can not execute the following SQL: 
DROP TABLE user_mwidmaie.stellar_masses_and_radii
Because: ERROR: table "stellar_masses_and_radii" does not exist